# Chapter 13 - Naive Bayes

This chapter is about starting naively to reduce bias and get the right answer from a Bayesian perspective. Worth reading the spam filter intro about Bayesian formulation and assumptions.

...and underflow of floats...

In [1]:
# bringing in old functions from previous chapters

import re
import math


In [3]:
# create a classifier

def tokenize(message):
    message = message.lower()
    all_words = re.findall("[a-z0-9']+", message) # extract the words...
    return set(all_words)

# count the words in a message
def count_words(training_se):
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] +=1
        return counts
    
# estimate probabilities using some smoothing from the spam/nonspam counts

def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    return [(w, (spam + k) / (total_spams + 2 * k), (non_spam + k) / (total_non_spams + 2*k)) 
            for w, (spam, non_spam) in counts.items()]
# nice one liner

def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        if word in maessage_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
            
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
            


In [5]:
# let's make a Class classifier

class NaiveBayesClassifier:
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
        
    def train(self, training_set):
        num_spams = len([is_spam for message, is_spam in training_set if i-_spam])
        num_non_spams = len(training_set) - num_spams
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts, num_spams, num_non_spams, self.k)
        
    def classify(self, message):
        return spam_probability(self.word_probs, message)
    
  

In [ ]:
# lets test it!

